In [ ]:
from matipo import SEQUENCE_DIR, GLOBALS_DIR
from matipo.sequence import Sequence
from matipo.util import ilt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.colorbar import Colorbar
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pathlib import Path

# progress_handler for Sequence.run() that simply prints the progress
def print_progress(p, l):
    print(p, '/', l)

# load SRCPMG pulse sequence from local directory
seq = Sequence('./SRCPMG.py')

##### SRCPMG Pulse Sequence
(90 pulse | `t_sat_i`) for `t_sat_i` in `t_sat` | `t_rec` | 90 pulse | `t_echo/2` | (180 pulse | `t_echo`) * `n_echo` | `t_end`

The saturation module constists of `len(t_sat)+1` pulses, so `t_sat` must be of even length to produce an odd number of 90 pulses.

From liturature, it's recommended that the values in `t_sat` are greater than the T2* of the magnet, and are non-regularly spaced with decreasing spacings.

Measure long `t_rec` first and short `t_rec` last to achieve optimal saturation at short recovery times.

Acquisitions are centred between 180 pulses.

In [ ]:
from matipo.util import flint

# modified from IRCPMG_T1T2_spectrum() in ilt.py to use the saturation recovery kernel function for T1
def SRCPMG_T1T2_spectrum(T1, T2, t_rec, t_echo, data, alpha=1, **kwargs):
    """
    SRCPMG_T1T2_spectrum: Calculate T1-T2 spectrum from Saturation Recovery CPMG data
    
    Parameters
    ----------
    T1 : float array
        T1 values for the output spectrum
    T2 : float array
        T2 values for the output spectrum
    t_rec : float array
        recovery times (seconds)
    t_echo : float or float array
        scalar echo spacing or array of echo times (seconds)
    data : 2D float or complex float array
        2D array of IRCPMG data, shape (number of inversion times, number of echos).
        The real part after autophasing will be used if complex.
    """
    
    N1, N2 = data.shape
    if len(t_rec) != N1:
        raise Exception(f'size of t_rec ({len(t_rec)} must match axis 0 of data ({N1}))')
    if np.isscalar(t_echo):
        t_echo = np.linspace(0, N2*t_echo, N2, endpoint=False)
    if len(t_echo) != N2:
        raise Exception(f'size of t_echo ({len(t_echo)} must match axis 1 of data ({N2}))')
    
    if np.any(np.iscomplex(data)):
        # autophase using first 2 points
        phase = np.angle(np.mean(data[0][:2]))
        y = np.real(data * np.exp(1j*-phase))
    else:
        y = data.copy()
    
    K1 = 1 - np.exp(-np.outer(t_rec,1/T1)) # T1 Saturation Recovery Kernel
    K2 = np.exp(-np.outer(t_echo,1/T2)) # T2 Kernel
    S, res = flint.flint(K1, K2, y, alpha, **kwargs)
    return S

In [ ]:
# set save directory and base file name
SAVE_DIR = '/home/data/SRCPMG'
SAVE_NAME = 'enter_name'

# make the save directory if it doesn't exist
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# load relevant global variables
seq.loadpar(GLOBALS_DIR+'frequency.yaml')
seq.loadpar(GLOBALS_DIR+'hardpulse_90.yaml')
seq.loadpar(GLOBALS_DIR+'hardpulse_180.yaml')

n_rec = 10
t_rec_min = 10e-3
t_rec_max = 100

# set some parameters directly (SI units)
seq.setpar(
    n_scans=4,
    n_echo=5000,
    t_echo=200e-6,
    n_samples=16,
    t_dw=4e-6,
    t_end=0.1, # don't need to wait for T1 recovery in saturation recovery type experiments, should still wait >>T2* to avoid coherence pathways
    t_sat=5e-3*np.array([7, 5, 3, 2]), # pulse spacings should be larger than T2* and decreasing (recommended by literature)
    t_rec=np.logspace(np.log10(t_rec_max), np.log10(t_rec_min), n_rec) # measure using recovery times in decreasing order (recommended by literature)
)

# print out the parameter set for reference
print(seq.par)

# run sequence, progress_handler is optional
await seq.run(progress_handler=print_progress)

# average the echos to get the integrated echo decay curve
y = np.reshape(np.mean(np.reshape(seq.data, (-1, seq.par.n_samples)), axis=1), (-1, seq.par.n_echo))

# correct phase assuming largest (first, using reverse order) recovery time has 0 phase
phase = np.angle(np.mean(y[0][:2]))
y *= np.exp(1j*-phase)

# save 2D data array in numpy format
np.save(f'{SAVE_DIR}/{SAVE_NAME}', y)
seq.savepar(f'{SAVE_DIR}/{SAVE_NAME}.yaml')

In [ ]:
# make time axes
t_rec = seq.par.t_rec
t_cpmg = np.linspace(0, seq.par.n_echo*seq.par.t_echo, seq.par.n_echo)

In [ ]:
plt.figure(figsize=(10, 5), dpi=100)
for i, t in enumerate(t_rec):
    plt.plot(t_cpmg, y[i].real, label=f't_inv: {t:.2E} s')
plt.ylabel('signal (V)')
plt.xlabel('time (s)')
plt.title('CPMG Decay')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# average first 2 points to get initial signal
plt.figure(figsize=(10, 5), dpi=100)
plt.semilogx(seq.par.t_rec, np.mean(y.real[:,:2], axis=1), label='real')
plt.semilogx(seq.par.t_rec, np.mean(y.imag[:,:2], axis=1), label='imag')
plt.ylabel('Initial Signal (V)')
plt.xlabel('Recovery Time (s)')
plt.title('Saturation Recovery')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Run Inverse Laplace Transform
n_T1 = 20
n_T2 = 20
alpha = 1
T1 = np.logspace(-3,1,n_T1)
T2 = np.logspace(-3,1,n_T2)
# using larger tolerance to speed up calculation, for better results
# transfer the data to a PC and run ILT with the default tolerance of 1e-5
S = SRCPMG_T1T2_spectrum(T1, T2, t_rec, t_cpmg, y, alpha, tol=1e-3, progress=1000)

# Plot the 2D T1-T2 map
x_ = np.log10(T2)
y_ = np.log10(T1)
x_lim = (x_.min(), x_.max())
y_lim = (y_.min(), y_.max())
fig= plt.figure(figsize=(10,10), dpi=100)
gs = gridspec.GridSpec(2, 2, width_ratios=[3,1], height_ratios=[1,3])
ax = plt.subplot(gs[1,0])
axr = plt.subplot(gs[1,1], sharey=ax)
axt = plt.subplot(gs[0,0], sharex=ax)
# main plot
ax.imshow(S, origin='lower', extent=x_lim+y_lim, aspect='auto', cmap='gray', interpolation='lanczos')
plt.xlim(x_lim)
ax.set_xlabel('log10(T2)')
ax.set_ylabel('log10(T1)')
# T1 projection plot
axr.plot(S.sum(1), y_)
axr.set_ylabel('log10(T1)')
# T2 projection plot
axt.plot(x_, S.sum(0))
axt.set_xlabel('log10(T2)')
plt.show()